# Model regresji prostej

$$Y = \beta_0 + \beta_1 X + \epsilon,$$
gdzie $\epsilon \sim \mathcal{N}(0, \sigma)$.

**Nomenklatura:**

$Y$ - zmienna zależna, zmienna odpowiedzi, zmienna objaśniana

$X$ - zmienna niezależna, predyktor, atrybut

$\beta_0, \beta_1$ - współczynniki regregresji

$\epsilon$ - błąd losowy

**Cel:** 

Przy użyciu par  $(x_1, y_1), (x_2, y_2), \ldots (x_n, y_n)$ odpowiadającym pomiarom, odpowiednio, 
zmiennej niezależnej $X$ i zmiennej zależnej $Y$, wyznaczyć współczynniki 
$b_0, b_1$ tak, aby 
$$y_i \approx b_0 + b_1 x_i$$

**Suma błędów  kwadratów:**
    
$$RSS = e_1^2 + e_2^2 + \ldots + e_n^2 = (y_1 - \hat{y_1})^2 + (y_2 - \hat{y_2})^2 + \ldots + (y_n - \hat{y_n})^2$$

**Funkcja kryterialna:**
    
$$(b_0, b_1) = \arg\!\min_{(b_0, b_1)} \sum_{i=1}^n (y_i - \hat{y_i})^2 = \arg\!\min_{(b_0, b_1)} \sum_{i=1}^n (y_i - (b_0 + b_1 x_i))^2$$

$$b_0 = \frac{\sum_{i=1}^{n}(x_i - \overline{x})(y_i - \overline{y})}{\sum_{i=1}^n(x_i - \overline{x})^2}$$

$$b_1 = \overline{y} - b_1\overline{x}$$

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from patsy import dmatrices
from statsmodels.stats.stattools import durbin_watson

import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Pakiet [`statsmodels`](https://www.statsmodels.org/stable/genindex.html)

In [ ]:
help(sm.datasets)

Help on package statsmodels.datasets in statsmodels:

NAME
    statsmodels.datasets - Datasets module

PACKAGE CONTENTS
    anes96 (package)
    cancer (package)
    ccard (package)
    china_smoking (package)
    co2 (package)
    committee (package)
    copper (package)
    cpunish (package)
    elnino (package)
    engel (package)
    fair (package)
    fertility (package)
    grunfeld (package)
    heart (package)
    interest_inflation (package)
    longley (package)
    macrodata (package)
    modechoice (package)
    nile (package)
    randhie (package)
    scotland (package)
    spector (package)
    stackloss (package)
    star98 (package)
    statecrime (package)
    strikes (package)
    sunspots (package)
    template_data
    tests (package)
    utils

FUNCTIONS
    check_internet(url=None)
        Check if internet is available
    
    clear_data_home(data_home=None)
        Delete all the content of the data home cache.
    
    get_data_home(data_home=None)
        Ret

In [ ]:
#pobieranie plików z statsmodels
cancer = sm.datasets.cancer.load_pandas()

In [ ]:
print(sm.datasets.cancer.DESCRSHORT)
print(sm.datasets.cancer.DESCRLONG)
print(sm.datasets.cancer.NOTE)

Breast Cancer and county population
The number of breast cancer observances in various counties
::

    Number of observations: 301
    Number of variables: 2
    Variable name definitions:

        cancer - The number of breast cancer observances
        population - The population of the county




In [ ]:
cancer.data.head()

,cancer,population
0,1.0,445.0
1,0.0,559.0
2,3.0,677.0
3,4.0,681.0
4,3.0,746.0


In [ ]:
# Pobieranie zbiorow danych z R

In [ ]:
carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [ ]:
#opis
print(carseats.__doc__)

.. container::

   ======== ===============
   Carseats R Documentation
   ======== ===============

   .. rubric:: Sales of Child Car Seats
      :name: sales-of-child-car-seats

   .. rubric:: Description
      :name: description

   A simulated data set containing sales of child car seats at 400
   different stores.

   .. rubric:: Usage
      :name: usage

   ::

      Carseats

   .. rubric:: Format
      :name: format

   A data frame with 400 observations on the following 11 variables.

   ``Sales``
      Unit sales (in thousands) at each location

   ``CompPrice``
      Price charged by competitor at each location

   ``Income``
      Community income level (in thousands of dollars)

   ``Advertising``
      Local advertising budget for company at each location (in
      thousands of dollars)

   ``Population``
      Population size in region (in thousands)

   ``Price``
      Price company charges for car seats at each site

   ``ShelveLoc``
      A factor with levels ``Bad``,

# Zadanie 1

Wczytaj zbiór `Carseats`, który zawierają dane o sprzedaży fotelików samochodowych pewnej firmy w 400 różnych lokalizacjach:

• sprzedaż jednostek (w tysiącach) w danej lokalizacji (`Sales`),

• cena sprzedaży konkurenta w danej lokalizacji (`CompPrice`),

• poziom dochodów społeczności wdanej lokalizacji w tys. dolarów (`Income`),

• wielkość populacji w danym regionie (`Population`),

• lokalny budżet reklamowy dla firmy w danej lokalizacji (`Advertising`),

• cena sprzedaży w danej lokalizacji (`Price`),

• na ile korzystne jest miejsce na półkach sklepu dla fotelików w danym regionie?
(`Shelveloc`),

• średni wiek ludności lokalnej (`Age`),

• poziom wykształcenia w każdej lokalizacji (`Education`),

• czy położenie sklepu jest w lokalizacji miejskiej? (`Urban`)

• czy sklep znajduje się w USA? (`US`)

1. Interesuje nas zależność wielkości sprzedaży fotelików samochodowych (w tys. jednostek) od ich ceny sprzedaży (zmienne `Sales` i `Price`). Określ zmienną zależną i niezależną, a następnie:
* Narysuj wykres rozproszenia tych zmiennych. 
* Czy zmienne te są zależne? 
* Oblicz dla nich współczynnik korelacji Pearsona. 
       
2. Wyznacz funkcję regresji wielkości sprzedaży fotelików względem ich ceny. Narysuj ją na wykresie rozproszenia.

3. Zweryfikuj dopasowanie modelu:
       a) Czy istnieje związek pomiędzy zmienną zależną a niezależną?
       b) Czy wyraz wolny jest istotny w modelu?
       c) Oblicz i zinterpretuj wartość współczynnika determinacji R2.
       d) Sprawdź, czy spełnione jest założenie o rozkładzie normalnym reszt.
       
4. Powtórz punkty 1-3 dla zależności wielkości sprzedaży fotelików samochodowych (w tys. jednostek) od ich ceny sprzedaży u konkurenta w danej lokalizacji (zmienne `Sales` i `CompPrice`).

In [ ]:
#Jak wyciągać poszczególne informacje z summary?
#print("Liczba obserwacji w zbiorze treningowym (No. Observations): {}".format(model3.nobs))
#print("\n")

#print("Liczba stopni swobody (Df Residuals): {}".format(model3.df_resid))
#print("Liczba stopni swobody w modelu (Df Model): {}".format(model3.df_model))
#print("\n")

#print("Współczynnik determinacji R2 (R-squared): {}".format(fitted_model3.rsquared))
#print("Modyfikowany współczynnik determinacji R2 (Adj. R-squared): {}".format(fitted_model3.rsquared_adj))
#print("\n")

#print("Wartość statystyki testowej testu F (F-statistic): {}".format(fitted_model3.fvalue))
#print("P-wartość testu F (Prob (F-statistic)): {}".format(fitted_model3.f_pvalue))
#print("\n")

#print("Wartość funkcji wiarogodności (Log-Likelihood): {}".format(fitted_model3.llf))
#print("AIC: {}".format(fitted_model3.aic))
#print("BIC: {}".format(fitted_model3.bic))
#print("\n")

#normaltest = stats.normaltest(fitted_model3.resid)
#print("Wartość statystyki testowej testu normalności D'Angostino (Omnibus): {}". format(normaltest.statistic))
#print("P-wartość testu D'Angostino (Prob(Omnibus)): {}".format(normaltest.pvalue))
#print("\n")

#print("Skośność (Skew): {}".format(stats.skew(fitted_model3.resid)))
#print("Kurtoza (Kurtosis): {}".format(stats.kurtosis(fitted_model3.resid, fisher = False, bias = True)))
#print("\n")

#normaltest2 = stats.jarque_bera(fitted_model3.resid)
#print("Wartość statystyki Durbina-Watsona: {}".format(durbin_watson(fitted_model3.resid)))
#print("Wartość statystyki Jarque-Bera: {}".format(normaltest2[0]))
#print("P-wartość testu Jarque-Bera: {}".format(normaltest2[1]))
#print("Wskaźnik uwarunkowania (Cond. No): {}".format(fitted_model3.condition_number))
#print("\n")

#print("Wartości dopasowanych wartości parametrów regresji:")
#print(fitted_model3.params)
#print("\n")

#print("Odchylenie standardowe dla dopasowanych estymatorów parametrów regresji:")
#print(fitted_model3.HC0_se)
#print("\n")

#print("Wartość statystki testu t-Studenta")
#print(fitted_model3.tvalues)
#print("\n")

#print("P-wartości testu t-Studenta")
#print(fitted_model3.pvalues)
#print("\n")

#print("Przedział ufności dla współczynnika regresji dla wyrazu wolnego:")
#print(fitted_model3.conf_int_el(0))
#print("Przedział ufności dla współczynnika regresji dla zmiennej TV:")
#print(fitted_model3.conf_int_el(1))

In [ ]:
#3
#print("Wartość statystki testu t-Studenta")
#print(fitted_model3.tvalues)
#print("\n")
#print("P-wartości testu t-Studenta")
#print(fitted_model3.pvalues)
#print("\n")
#print("Współczynnik determinacji R2 (R-squared): {}".format(fitted_model3.rsquared)) 
#print("\n")
#normaltest = stats.normaltest(fitted_model3.resid)
#print("Wartość statystyki testowej testu normalności D'Angostino (Omnibus): {}". format(normaltest.statistic))
#print("P-wartość testu D'Angostino (Prob(Omnibus)): {}".format(normaltest.pvalue))
#print("\n")
#normaltest2 = stats.jarque_bera(fitted_model3.resid)
#print("Wartość statystyki Jarque-Bera: {}".format(normaltest2[0]))
#print("P-wartość testu Jarque-Bera: {}".format(normaltest2[1]))

# Przekształcanie zmiennej objaśniającej

# Zadanie 2
Wczytaj dane `pollution.csv` (opis danych: https://www.rdocumentation.org/packages/SMPracticals/versions/1.4-3/topics/pollution). Dopasuj model regresji liniowej postaci: `mort~nox`. Czy model jest dobrze dopasowany do danych? Jeżeli nie, dlaczego?

In [ ]:
#https://www.rdocumentation.org/packages/SMPracticals/versions/1.4-3/topics/pollution
pollution = pd.read_csv("pollution.csv")

### Metoda Boxa-Coxa

Metoda Boxa-Coxa jest popularnym rozwiązaniem stosowanym do przekształcenia zmiennej. Ważnym założeniem metody jest dodatniość wszystkich wartość.
Metoda przekształca zmienną $z>0 \to t_{\lambda}(z)$, gdzie 
$$
    t_{\lambda}(z)=\left.
    \begin{cases}
    \frac{z^{\lambda}-1}{\lambda} \quad &\text{dla} \quad \lambda\neq 0\\
    \log(z) \quad &\text{dla} \quad \lambda= 0.
    \end{cases}\right.
$$

Wybór $\lambda$ odbywa się na podstawie metody największej wiarogodności. Funkcja wiarogodności zakłada normalność błędów i ma następującą postać:
$$
L(\lambda)=-\frac{n}{2}\log\Big(\frac{\text{RRS}_{\lambda}}{n}\Big) + (\lambda-1)\sum\log y_i,
$$
gdzie $\text{RRS}_{\lambda}$, to $\text{RRS}$ dla odpowiedzi $t_{\lambda}(z)$.

## Zadanie 2.1
Znajdź najlepszy parametr $\lambda$ dla przekształcenia zmiennej niezależnej `nox`, przekształć zmienną zgodnie z uzyskaną wartością i dopasuj model prostej regresji liniowej.

Co w przypadku, gdy zmienna losowa, która chcielibyśmy przekształcić byłaby niedodatnia?

### Metoda Yeo-Johnsona

$$
    t_{\lambda}(z)=\left.
    \begin{cases}
    \frac{(y+1)^{\lambda} -1}{\lambda} \quad &\text{dla} \quad \lambda\neq 0, y\geq 0,\\
    \log(y+1) \quad &\text{dla} \quad \lambda= 0, y\geq0,\\
    -\frac{(-y+1)^{2-\lambda}-1}{2-\lambda} \quad &\text{dla} \quad \lambda\neq 2, y<0,\\
    -\log(-y+1) \quad &\text{dla} \quad \lambda= 2, y<0.
    \end{cases}\right.
$$

In [ ]:
?stats.yeojohnson

# Zmienne kategoryczne

# Zadanie 3
Wczytaj dane `Salaries` z pakieru `carData` i dopasuj model `salary ~ sex`. Zwróć uwagę na kodowanie zmiennej `sex`. 

In [ ]:
# http://www.sthda.com/english/articles/40-regression-analysis/163-regression-with-categorical-variables-dummy-coding-essentials-in-r/
salaries = sm.datasets.get_rdataset(dataname="Salaries", package="carData")
salaries.data.head()

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500


**Ważne:**

Gdy w danych występują zmienne kategoryczne, jak np. w tym przypadku zmienna `sex`, postępujemy z nią następująco

1. tworzymy, nową zmienną (ang. *dummy variable*), która

    **1**, gdy osoba jest mężczyzną,
    
    **0**, gdy osoba jest kobietą,
    
    
2. zmienna `sex` jest traktowana w równaniu regresji jako:

  $b_0 + b_1$ gdy osoba jest mężczyzną,

  $b_0$ gdy osoba jest kobietą.

teraz zmieniamy poziom referencyjny aby był nim Male wówczas zmienna sex jest traktowana w równaniu regresji jako:\
$c_0$ gdy osoba jest mężczyzną,\
$c_0+ c_1$ gdy osoba jest kobietą.\
Zatem $c_0 = b_0+b_1$ i $c_1 = b_0-c_0 = -b_1$

## Zadanie 3.1 
Sprawdź jak została zakodowana zmienna `ShelveLoc` w zbiorze danych `Carseats`. Zmień poziom wartości dla tej zmiennej, sprawdź różnice w modelu.

# Zadanie 4
Napisz funkcję, która dla wybranej zmiennej niezależnej dopasuje model prostej liniowej postaci: `Sales ~ X`, gdzie `X` to wybrana zmienna. Następnie, korzystając z napisanej funkcji dopasuj model do każdej zmiennej niezależnej w zbiorze `Carseats`.

In [ ]:
#def XY_generator(col):
#  return(dmatrices('Sales ~ {}'.format(col), data=carseats.data, return_type='dataframe'))

#plt.figure(figsize=(20, 40))

#columns = carseats_df.columns[carseats_df.columns != 'Sales']

#for i, col in enumerate(columns):
#    #dopasowujemy dla zmiennej zależnej Sales  jedną kolumnę z danych (w każdym obrocie inną)
#    Y, X = XY_generator(col)

#    plt.subplot(len(columns) // 2 + 1, 2, i+1)
#    #rysujemy wykres rozproszenia dla każdej pary zmienna niezależna vs zmienna zależna
#    plt.scatter(carseats_df[col], Y, marker='o')
    
#    #dopasowujemy model jednej zmiennej
#    model = sm.OLS(endog=Y, exog=X)
#    fitted_model = model.fit()
#    plt.plot(carseats_df[col], fitted_model.fittedvalues, color='red')
    
#    plt.xlabel(col)
#    plt.ylabel('Sales')
#    #dopisujemy charakterystyki danych modeli R^2 im większe tym lepiej,
#    #AIC i BIC im mniej tym lepiej
#    plt.title("R2={}, AIC={}, BIC={}".format(np.round(fitted_model.rsquared, 2), 
#                                            np.round(fitted_model.aic, 2),
#                                            np.round(fitted_model.bic, 2)))